### A BERTopic-ról
https://maartengr.github.io/BERTopic/algorithm/algorithm.html#6-optional-fine-tune-topic-representation

https://towardsdatascience.com/topics-per-class-using-bertopic-252314f2640

https://people.inf.elte.hu/csa/html/szinek.htm

In [ ]:
from bertopic import BERTopic
from hdbscan import HDBSCAN
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance
from sentence_transformers import SentenceTransformer

import pickle
import huspacy


In [ ]:
meta = pickle.load(open("../data/meta.pkl", "rb"))
lemmatized = pickle.load(open("../data/lemmatized.pkl", "rb"))
#pos = pickle.load(open("../data/pos.pkl", "rb"))
#tokens =  pickle.load(open("../data/tokenized.pkl", "rb"))
#doc_stop = pickle.load(open("../data/no_stopword.pkl", "rb")) ### kevesebb stopszóval
doc_stop_2 = pickle.load(open("../data/stopword_filtered.pkl", "rb")) ### több stop szóval
all_docs = pickle.load(open("../data/docs.pkl", "rb"))

## 1. Betöltjük az általunk választott modellt és ábrázoljuk az eredeti beszélgetések topikjait. Több lementett modellel is kísérletezhetünk.

In [ ]:
loaded_model_path = "../data/models/bert_stop_2_probabilities_merged_85"
#esetleg: "bert_model_reduced_prob" vagy bert_stop_new: ez új stopszó listával vagy bert_stop_distribution
loaded_model = BERTopic.load(loaded_model_path)
print("A modellt betöltöttük")

In [ ]:
loaded_model.get_topic_info()

In [ ]:
t = loaded_model.get_topic_info()
print(t)

## 2. Megcsináljuk a html-t

In [ ]:
html_start = """<!DOCTYPE html>
<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1"  charset="UTF-8">
<style>
div.sticky {
  position: sticky; top:30%;
  background-color: white;
  padding: 50px;
  font-size: 10px; float: right; display:table;
}
table {
    border-collapse: collapse;
    width: 100%;
}
th, td {
    border: 1px solid #7a3f3f;
    padding: 10px;
    text-align: center;
}
</style>
</head>
<body>

<div class="sticky">
    <table>
        <tr>
            <td>Téma</td>
            <td>Szín</td>
        </tr>"""

In [ ]:
legend_end = """
</table>
</div>
"""

In [ ]:
html_end = """
</body>
</html>
"""

## 3. Összerakjuk az eredeti dialógusokat és a beszédfordulókhoz rendeljük a legvalószínűbb (első helyen álló) topikját, megszínezzük. Egy mappába lementjük az összes html file-t

In [ ]:
import os

main_topics = loaded_model.topics_
with open("../data/colors.txt", "r") as infile:
    colors = infile.read().strip().split("\n")
ddir = "../data/combined_transcripts_freeConv"
input_files = [f for f in os.listdir(ddir) if os.path.isfile(os.path.join(ddir, f))]
meta2idx = {v: k for k,v in meta.items()}
topic_num2name = dict(zip(t["Topic"], t["Name"]))

for input_file in input_files:
    legend_table = []
    local_html = html_start
    # html eleje
    with open(os.path.join(ddir, input_file)) as infile:
        txt = infile.read()
    # találd ki hogy mordor vagy gondor az első
    dialogue_docs = [d for d in meta.values() if d.startswith(input_file)] # a dialógushoz tartozó dokumentumok listája
    dialogue_idx = [meta2idx[d] for d in dialogue_docs] # a dialógushoz tartozó dokumentumok indexeinek listája

    mordor = [d for d in dialogue_docs if "mordor" in d] # a Mordor nevű karakterhez tartozó dokumentumok listája
    mordor_idx = [meta2idx[d] for d in mordor] # a Mordor nevű karakterhez tartozó dokumentumok indexeinek listája
    mordor_topics = [main_topics[i] for i in mordor_idx] # a Mordor nevű karakterhez tartozó dokumentumok témáinak listája
    gondor = [d for d in dialogue_docs if "gondor" in d] # a Gondor nevű karakterhez tartozó dokumentumok listája
    gondor_idx = [meta2idx[d] for d in gondor] # a Gondor nevű karakterhez tartozó dokumentumok indexeinek listája
    gondor_topics = [main_topics[i] for i in gondor_idx] # a Gondor nevű karakterhez tartozó dokumentumok témáinak listája

    if txt.startswith("Mordor:"):
        zipped_idx = list(zip(mordor_idx, gondor_idx))
    else:
        zipped_idx = list(zip(gondor_idx, mordor_idx))

    number_of_topics = list(set(gondor_topics + mordor_topics)) # a dialógusban szereplő témák számának meghatározása
    topic2color = {i: colors[i] for i in number_of_topics}
    for k, v in topic2color.items():
        print(v)
        row = f"""<tr>
                <td>{topic_num2name[k]}</td>
                <td style="background-color: {v}"></td>
            </tr>\n"""
        local_html += row
    local_html += legend_end
    for e in zipped_idx:
        turn1 = all_docs[e[0]]
        turn2 = all_docs[e[1]]
        topic1 = main_topics[e[0]]
        topic2 = main_topics[e[1]]

    o = []
    for e in zipped_idx:
        turn1 = all_docs[e[0]].replace("<", "").replace(">", "")
        turn2 = all_docs[e[1]].replace("<", "").replace(">", "")

        topic1 = main_topics[e[0]]
        topic2 = main_topics[e[1]]

        html1 = f"<p style=\"background-color:{topic2color[topic1]};\">{turn1}</p>"
        html2 = f"<p style=\"background-color:{topic2color[topic2]};\">{turn2}</p>"
        o.append(html1)
        o.append(html2)
    local_html += "\n".join(o)
    final_html = local_html + html_end
    with open(f"../data/htmls/{input_file}.html", "w") as outfile:
        outfile.write(final_html)